<a href="https://colab.research.google.com/github/praveen1608/Praveen-Reddy_INFO5731_Spring2024/blob/main/INFO_5731_Project_Code_part_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Import packages**

In [ ]:
!pip install bert-score
!pip install -U spacy
!pip install gensim

# Download spaCy model for word embeddings
!python -m spacy download en_core_web_md

import spacy
# Load spaCy model with word vectors
nlp = spacy.load('en_core_web_md')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 11.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
# Installing Gemini API using pip.

!pip install -q -U google-generativeai

# Importing required packages and converting input text to Markdown format (lightweight markup language with plain-text formatting syntax  to create rich text using a simple and easy-to-read syntax)

import pathlib
import textwrap

import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

Setup your API key
Before we use the Gemini API, we must first obtain an API key. We created the key using Google AI Studio.

Get an API key.

In [ ]:
# Used to securely store the API key
from google.colab import userdata

#Created a API key using google account and added the key to this project
#Or use `os.getenv('GOOGLE_API_KEY')` to fetch an environment variable.

GOOGLE_API_KEY=userdata.get('gemini_key')

genai.configure(api_key=GOOGLE_API_KEY)

In Colab, add the key to the secrets manager under the "🔑" in the left panel. Give it the name GOOGLE_API_KEY.

# List models
Now we'll call the Gemini API. We will use list_models to see the available Gemini models:

*   gemini-pro: optimized for text-only prompts.
*   gemini-pro-vision: optimized for text-and-images prompts



In [ ]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-1.5-pro-latest
models/gemini-pro
models/gemini-pro-vision


**Table bold text with Statistics vs Table with text Summary**

Gemini provides a multimodal model (gemini-pro-vision) that accepts both text and images and inputs. The GenerativeModel.generate_content API is designed to handle multimodal prompts and returns a text output.

In [ ]:
# Main source code to generate AI summary for each image

import os
import pandas as pd
import zipfile
from PIL import Image
import google.generativeai as genai

def generate_image_summaries_to_dataframe(zip_file_path):
  """Generates summaries for images within a zip folder and creates a Pandas DataFrame.

  Args:
      zip_file_path (str): Path to the zip folder containing images.

  Returns:
      pd.DataFrame: DataFrame with columns 'Image Name' and 'Summary'.
  """

  data = []
  with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    for image_file in zip_ref.namelist():
      if image_file.lower().endswith(('.jpg', '.jpeg', '.png')):
        # Extract image from zip
        zip_ref.extract(image_file)

        # Load image
        img = Image.open(image_file)

        # Generate summary using GenAI (assuming model is loaded)
        model = genai.GenerativeModel('gemini-pro-vision')
        response = model.generate_content(img)
        #summary = response.text
        if response.parts:
          summary = response.text
        else:
          # Handle invalid response
          summary = "Error: Invalid response from model"

        # Append data for DataFrame
        data.append({'Image Name': image_file, 'AI generated Summary': summary})

        # Delete extracted image
        os.remove(image_file)

  # Create DataFrame
  df = pd.DataFrame(data)
  return df

# Example usage
zip_file_path = '/content/Tables_PraveenReddy_zip.zip'
# Replace with your zip file path
text_stat_comp = generate_image_summaries_to_dataframe(zip_file_path)
text_stat_comp.head(6)

,Image Name,AI generated Summary
0,01.jpg,The table shows the number of posts and comme...
1,02.jpg,NSW Health has been made aware of a social me...
2,03.jpg,The table shows the role of interviewees and ...
3,04.jpg,The table shows the percentage of people in e...
4,05.jpg,The table shows the results of a structural e...
5,06.jpg,The table shows the data for the variables us...


In [ ]:
#Reading author summary example file

Stats_text_comp = pd.read_csv('/content/Tables_AuthorSummary_PraveenReddy.csv', encoding='latin-1')

#Joining Author summary Example csv with AI generated summary Examples
text_stat_comp_1 = text_stat_comp.reset_index()
Stats_text_comp_final = text_stat_comp_1.merge(Stats_text_comp, on = 'index')
Stats_text_comp_final.head(4)

,index,Image Name,AI generated Summary,Author Summary
0,0,01.jpg,The table shows the number of posts and comme...,We used a self-developed Python app and collec...
1,1,02.jpg,NSW Health has been made aware of a social me...,To investigate official Facebook use against m...
2,2,03.jpg,The table shows the role of interviewees and ...,We conducted nineteen semi-structured intervie...
3,3,04.jpg,The table shows the percentage of people in e...,"A total of 120 participants (Mage = 30.98, SDa..."


**Basic Cosine Similarity**

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

def preprocess_text(text):
  """Preprocesses text for similarity comparison."""
  text = text.lower()
  text = ''.join([c for c in text if c.isalnum() or c.isspace()])  # Remove punctuation
  stop_words = stopwords.words('english')
  words = [w for w in text.split() if w not in stop_words]
  stemmer = PorterStemmer()
  stemmed_words = [stemmer.stem(w) for w in words]
  return stemmed_words

def calculate_similarity(summary1, summary2, metric='cosine'):
  """Calculates similarity between summaries using chosen metric."""
  if metric == 'cosine':
    vectorizer = TfidfVectorizer()
    vectors = vectorizer.fit_transform([summary1, summary2])
    return vectors.toarray().dot(vectors.toarray().T)[0, 1]
  elif metric == 'jaccard':
    summary1_words = set(preprocess_text(summary1))
    summary2_words = set(preprocess_text(summary2))
    intersection = len(summary1_words.intersection(summary2_words))
    union = len(summary1_words.union(summary2_words))
    return intersection / union if union else 0
  else:
    raise ValueError("Invalid metric. Choose 'cosine' or 'jaccard'.")

# Choose similarity metric (cosine or jaccard)
metric = 'cosine'

Stats_text_comp_final['Cosine_Similarity Score'] = Stats_text_comp_final.apply(lambda row: calculate_similarity(row['AI generated Summary'], row['Author Summary'], metric), axis=1)

Stats_text_comp_final.head(5)

,index,Image Name,AI generated Summary,Author Summary,Cosine_Similarity Score
0,0,01.jpg,The table shows the number of posts and comme...,We used a self-developed Python app and collec...,0.364668
1,1,02.jpg,NSW Health has been made aware of a social me...,To investigate official Facebook use against m...,0.074238
2,2,03.jpg,The table shows the role of interviewees and ...,We conducted nineteen semi-structured intervie...,0.404120
3,3,04.jpg,The table shows the percentage of people in e...,"A total of 120 participants (Mage = 30.98, SDa...",0.245916
4,4,05.jpg,The table shows the results of a structural e...,A one-way ANOVA was used for the manipulation ...,0.366502


**Rouge-WE**

In [ ]:
import spacy
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def calculate_rouge_we(ai_summary, author_summary):
    # Tokenize and process AI-generated summary
    ai_doc = nlp(ai_summary)
    ai_tokens = [token for token in ai_doc if not token.is_stop]

    # Tokenize and process Author summary
    author_doc = nlp(author_summary)
    author_tokens = [token for token in author_doc if not token.is_stop]

    # Calculate word embeddings for AI-generated summary
    ai_vec = np.mean([token.vector for token in ai_tokens], axis=0).reshape(1, -1)

    # Calculate word embeddings for Author summary
    author_vec = np.mean([token.vector for token in author_tokens], axis=0).reshape(1, -1)

    # Compute cosine similarity between word embeddings
    similarity_score = cosine_similarity(ai_vec, author_vec)[0][0]

    return similarity_score

# Apply calculate_rouge_we function to DataFrame
Stats_text_comp_final['ROUGE-WE'] = Stats_text_comp_final.apply(lambda row: calculate_rouge_we(row['AI generated Summary'], row['Author Summary']), axis=1)

# Display DataFrame with ROUGE-WE scores
Stats_text_comp_final.head(6)

,index,Image Name,AI generated Summary,Author Summary,Cosine_Similarity Score,ROUGE-WE
0,0,01.jpg,The table shows the number of posts and comme...,We used a self-developed Python app and collec...,0.364668,0.788829
1,1,02.jpg,NSW Health has been made aware of a social me...,To investigate official Facebook use against m...,0.074238,0.794754
2,2,03.jpg,The table shows the role of interviewees and ...,We conducted nineteen semi-structured intervie...,0.404120,0.928972
3,3,04.jpg,The table shows the percentage of people in e...,"A total of 120 participants (Mage = 30.98, SDa...",0.245916,0.755894
4,4,05.jpg,The table shows the results of a structural e...,A one-way ANOVA was used for the manipulation ...,0.366502,0.898678
5,5,06.jpg,The table shows the data for the variables us...,Our study used stratified random sampling of 9...,0.452996,0.911058


**Bert Score**

In [ ]:
import pandas as pd
from bert_score import score

def calculate_bert_score(ai_summary, author_summary):
    # Compute BERTScore for the summaries
    P, R, F1 = score([ai_summary], [author_summary], lang='en', verbose=False)

    # Extract the F1 score (you can use other scores like P or R as needed)
    bert_score = F1.item()

    return bert_score

# Apply calculate_bert_score function to DataFrame
Stats_text_comp_final['BertScore'] = Stats_text_comp_final.apply(lambda row: calculate_bert_score(row['AI generated Summary'], row['Author Summary']), axis=1)

# Display DataFrame with BertScore values
Stats_text_comp_final.head(5)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['ro

,index,Image Name,AI generated Summary,Author Summary,Cosine_Similarity Score,ROUGE-WE,BertScore
0,0,01.jpg,The table shows the number of posts and comme...,We used a self-developed Python app and collec...,0.364668,0.788829,0.855984
1,1,02.jpg,NSW Health has been made aware of a social me...,To investigate official Facebook use against m...,0.074238,0.794754,0.816001
2,2,03.jpg,The table shows the role of interviewees and ...,We conducted nineteen semi-structured intervie...,0.404120,0.928972,0.843735
3,3,04.jpg,The table shows the percentage of people in e...,"A total of 120 participants (Mage = 30.98, SDa...",0.245916,0.755894,0.808329
4,4,05.jpg,The table shows the results of a structural e...,A one-way ANOVA was used for the manipulation ...,0.366502,0.898678,0.804550


**Bleu**

In [ ]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import pandas as pd

def calculate_bleu_score(ai_summary, author_summary):
    # Tokenize the summaries into lists of words
    ai_tokens = ai_summary.split()
    author_tokens = author_summary.split()

    # Check if both summaries are non-empty
    if not ai_tokens or not author_tokens:
        return 0.0  # Return zero BLEU score for empty summaries

    # Use SmoothingFunction with Chen-Cherry method for BLEU score calculation
    smoothing = SmoothingFunction()
    # Calculate BLEU score with unigram (1-gram) precision, and Chen-Cherry smoothing
    bleu_score = sentence_bleu([author_tokens], ai_tokens, weights=(1,), smoothing_function=smoothing.method7)

    return bleu_score

# Apply calculate_bleu_score function to DataFrame
Stats_text_comp_final['BLEU Score'] = Stats_text_comp_final.apply(lambda row: calculate_bleu_score(row['AI generated Summary'], row['Author Summary']), axis=1)

# Display DataFrame with BLEU Score values
Stats_text_comp_final.head(5)

,index,Image Name,AI generated Summary,Author Summary,Cosine_Similarity Score,ROUGE-WE,BertScore,BLEU Score
0,0,01.jpg,The table shows the number of posts and comme...,We used a self-developed Python app and collec...,0.364668,0.788829,0.855984,0.518519
1,1,02.jpg,NSW Health has been made aware of a social me...,To investigate official Facebook use against m...,0.074238,0.794754,0.816001,0.114711
2,2,03.jpg,The table shows the role of interviewees and ...,We conducted nineteen semi-structured intervie...,0.404120,0.928972,0.843735,0.205404
3,3,04.jpg,The table shows the percentage of people in e...,"A total of 120 participants (Mage = 30.98, SDa...",0.245916,0.755894,0.808329,0.048310
4,4,05.jpg,The table shows the results of a structural e...,A one-way ANOVA was used for the manipulation ...,0.366502,0.898678,0.804550,0.423977


**Meteor Score**

In [ ]:
import pandas as pd
import re

def calculate_meteor_score(generated_summary, reference_summary):
    """
    Calculates the METEOR score between a generated summary and a reference summary.

    Args:
        generated_summary (str): The generated summary.
        reference_summary (str): The reference summary.

    Returns:
        float: The METEOR score between 0 and 1.
    """
    # Preprocess text by removing punctuation and converting to lowercase
    generated_summary = re.sub(r"[^\w\s]", "", generated_summary.lower())
    reference_summary = re.sub(r"[^\w\s]", "", reference_summary.lower())

    # Split sentences into word lists
    generated_words = generated_summary.split()
    reference_words = reference_summary.split()

    # Calculate sentence-level METEOR scores
    meteor_score = 0
    for generated_sentence in generated_words:
        max_overlap = 0
        for reference_sentence in reference_words:
            overlap = min(len(generated_sentence), len(reference_sentence)) - (
                len(generated_sentence) - len(set(generated_sentence).intersection(reference_sentence))
            )
            max_overlap = max(max_overlap, overlap)
        meteor_score += max_overlap / len(generated_sentence)

    # Calculate final METEOR score (average across sentences)
    return meteor_score / len(generated_words)

# Add new column to store METEOR scores
Stats_text_comp_final["METEOR Score"] = None

# Calculate METEOR score for each pair of summaries
for index, row in Stats_text_comp_final.iterrows():
    generated_summary = row["AI generated Summary"]
    reference_summary = row["Author Summary"]
    meteor_score_value = calculate_meteor_score(generated_summary, reference_summary)
    Stats_text_comp_final.at[index, "METEOR Score"] = meteor_score_value


# Print the dataframe with the new column
Stats_text_comp_final.head(4)

,index,Image Name,AI generated Summary,Author Summary,Cosine_Similarity Score,ROUGE-WE,BertScore,BLEU Score,METEOR Score
0,0,01.jpg,The table shows the number of posts and comme...,We used a self-developed Python app and collec...,0.364668,0.788829,0.855984,0.518519,0.73399
1,1,02.jpg,NSW Health has been made aware of a social me...,To investigate official Facebook use against m...,0.074238,0.794754,0.816001,0.114711,0.838013
2,2,03.jpg,The table shows the role of interviewees and ...,We conducted nineteen semi-structured intervie...,0.404120,0.928972,0.843735,0.205404,0.85389
3,3,04.jpg,The table shows the percentage of people in e...,"A total of 120 participants (Mage = 30.98, SDa...",0.245916,0.755894,0.808329,0.048310,0.79718


In [ ]:
# Save the DataFrame to a CSV file
Stats_text_comp_final.to_csv('Tables_comparision_PraveenReddy.csv', index=False)


In [ ]:
from google.colab import files

In [ ]:
files.download('Tables_comparision_PraveenReddy.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>